In [1]:
# !pip install -U segmentation-models-pytorch albumentations --user
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
# import matplotlib.pyplot as plt
import linzhutils as lu
import tqdm
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [2]:
DATA_DIR = '/scratch/project_2007251/2019/hel2019/'

IMAGE_DIR = 'output'
img_dir = os.path.join(DATA_DIR, IMAGE_DIR)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

In [3]:
# load best saved checkpoint
model = torch.load('./models/kussi_FPN_efficientnet-b7_7_0.490.pth')
# ENCODER = 'efficientnet-b7'
# ENCODER_WEIGHTS = 'imagenet'
# CLASSES = ['kussi']
batch_size = 128
input_batch = []
mask_batch = []
result_masks = []
total_size = 0
file_num = 0

In [4]:
fileList=lu.getFileList(img_dir)
for i in range(len(fileList)):
    fileList[i] = os.path.join(img_dir, fileList[i])
fileList = sorted(fileList)
# np.save('/scratch/project_2007251/2019/results/file_list.npy',np.array(fileList))

In [5]:

def save_masks(masks):
    global file_num
    filename = f"/scratch/project_2007251/2019/results/result_masks_{file_num}.npy"
    concatenated = np.concatenate(masks, axis=0)
    reshaped = concatenated.reshape((-1, 512, 512))
    np.save(filename, np.array(masks))
    print(f"Saved {filename}")
    file_num += 1

for i in tqdm.tqdm(range(len(fileList))):
    image = cv2.imread(fileList[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_batch.append(to_tensor(image))

    if len(input_batch) == batch_size:
        x_tensor = torch.from_numpy(np.array(input_batch)).to(DEVICE)
        # print(x_tensor.shape)
        pr_mask_batch = model.predict(x_tensor).squeeze().cpu().numpy().round()
        result_masks.append(pr_mask_batch)
        total_size += pr_mask_batch.nbytes

        if total_size > 1e9:
            save_masks(result_masks)
            result_masks = []
            total_size = 0

        input_batch = []
        mask_batch = []
save_masks(result_masks) 

  1%|          | 533/50400 [00:27<42:35, 19.51it/s]  


KeyboardInterrupt: 

In [8]:
import matplotlib.pyplot as plt
# # helper function for data visualization
# def visualize(**images):
#     """PLot images in one row."""
#     n = len(images)
#     plt.figure(figsize=(16, 5))
#     for i, (name, image) in enumerate(images.items()):
#         plt.subplot(1, n, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.title(' '.join(name.split('_')).title())
#         plt.imshow(image)
#     plt.show()


In [35]:
import matplotlib.pyplot as plt
batch_size = 1
input_batch = []
mask_batch = []
images_list = []
result_masks = []
total_size = 0
file_num = 0
start_index = 10000

for i in tqdm.tqdm(range(start_index, 100000)):
    image = cv2.imread(fileList[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images_list.append(image)
    input_batch.append(to_tensor(image))

    # if len(input_batch) == batch_size:
    x_tensor = torch.from_numpy(np.array(input_batch)).to(DEVICE)
    pr_mask_batch = model.predict(x_tensor).squeeze().cpu().numpy().round()
    result_masks.append(pr_mask_batch)
    input_batch = []
        

  0%|          | 129/90000 [00:12<2:22:43, 10.50it/s]


In [1]:
for i in range(len(images_list)):
    plt.subplot(1, 2, 1)
    plt.imshow(images_list[i])
    plt.subplot(1, 2, 2)
    plt.imshow(result_masks[i])
    plt.show()
    # break

In [ ]:
for i in range(len(result_masks[0])):
    plt.figure()
    
    plt.imshow(result_masks[0][i])
    plt.show()

In [1]:
# import matplotlib.pyplot as plt
# import numpy as np

# # load the image and mask lists
# images = np.array(input_batch)
# masks = pr_mask_batch

# # plot the images and masks

# for i in range(images.shape[0]):
#     # create a figure with two subplots
#     fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
#     # plot the image
#     axs[0].imshow(images[i][0])
#     axs[0].axis('off')
#     axs[0].set_title('Image {}'.format(i+1))
    
#     # plot the mask
#     axs[1].imshow(masks[i]*255, cmap='gray')
#     axs[1].axis('off')
#     axs[1].set_title('Mask {}'.format(i+1))
    
#     # show the figure
#     plt.show()

In [24]:
# import time

# for i in range(20):
#     n = np.random.choice(len(test_dataset))

#     t1 = time.time()
    
#     image_vis = test_dataset_vis[n][0].astype('uint8')
#     image, gt_mask = test_dataset[n]

#     gt_mask = gt_mask.squeeze()

#     x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
#     pr_mask = model.predict(x_tensor)
#     pr_mask = (pr_mask.squeeze().cpu().numpy().round())

#     print(time.time()-t1)
    
#     visualize(image=image_vis,
#               ground_truth_mask=gt_mask,
#               predicted_mask=pr_mask)


NameError: name 'test_dataset' is not defined